In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
# load the dataset
df=pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocess the data
# drop irrelevant feature
df=df.drop(["RowNumber", "CustomerId", "Surname"], axis=1)

In [4]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Encode categorical variable
label_encoder_gender=LabelEncoder()
df["Gender"]=label_encoder_gender.fit_transform(df["Gender"])

In [6]:
df.head(
    
)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [9]:
# One hot encode gerography column
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geography=OneHotEncoder(sparse_output=False)
geo_encoder=one_hot_encoder_geography.fit_transform(df[["Geography"]])
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [11]:
one_hot_encoder_geography.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [12]:
# Converting sparse matric in the form of dataframe
geo_encoded_df= pd.DataFrame(geo_encoder, columns=one_hot_encoder_geography.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [13]:
## Combine one hot encoding column to original data
df=pd.concat([df.drop("Geography", axis=1), geo_encoded_df], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [14]:
# Save the encoders and scalar
with open("label_encoder_gender.pkl", "wb") as file:
    pickle.dump(label_encoder_gender, file)

with open("one_hot_encoder_geography.pkl", "wb") as file:
    pickle.dump(one_hot_encoder_geography, file)

In [15]:
# Divide the dataset into independent and dependent feature
X=df.drop("Exited", axis=1)
y=df["Exited"]

# Split the data in test and train
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

# Scale the features
scalar=StandardScaler()
X_train=scalar.fit_transform(X_train)
X_test=scalar.transform(X_test)

In [16]:
with open("scalar.pkl", "wb") as file:
    pickle.dump(scalar, file)

In [17]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


## ANN Implementation

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [20]:
# Build our ANN model
model=Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)), ## HL1 connected to inpout layer
    Dense(32, activation="relu"), ## HL2
    Dense(1, activation="sigmoid") ## Output layer
])

/Users/chandanjha/Documents/ann_classification/cenv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Another way of using optimizers
opt=tf.keras.optimizers.Adam(learning_rate=0.01)

In [23]:
# Another way of providing loss
loss=tf.keras.losses.BinaryCrossentropy()

In [24]:
## Compile the model
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Setup the tensorboard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [28]:
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [29]:
# Setup early stopping
# after some epoch the loss value is not decreasing and then we donot need to run all the epochs
# At that time we stopped the iterations
early_stopping_callback=EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

In [30]:
## train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7978 - loss: 0.4471 - val_accuracy: 0.8470 - val_loss: 0.3648
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - accuracy: 0.8535 - loss: 0.3542 - val_accuracy: 0.8640 - val_loss: 0.3447
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step - accuracy: 0.8598 - loss: 0.3519 - val_accuracy: 0.8585 - val_loss: 0.3410
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - accuracy: 0.8676 - loss: 0.3365 - val_accuracy: 0.8515 - val_loss: 0.3658
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step - accuracy: 0.8669 - loss: 0.3289 - val_accuracy: 0.8605 - val_loss: 0.3576
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step - accuracy: 0.8659 - loss: 0.3305 - val_accuracy: 0.8615 - val_loss: 0.3421
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 883us/step - accuracy: 0.8601 - loss: 0.3398 - val_accuracy: 0.8580 - val_loss: 0.3439
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step - accuracy: 0.8603 - loss: 0.3

In [31]:
model.save("model.h5") # h5 file is compatible with keras

In [37]:
## load tesnor board extension
import tensorboard
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 54184), started 0:01:21 ago. (Use '!kill 54184' to kill it.)

In [ ]:
## load the pickle file